### Laboratório 4

Nesse laboratório, o objetivo será preparar um pequeno sistema de tradução. Para desenvolver esse sistema alguns conceitos serão trabalhados.

O primeiro deles é projeção. É possível projetar um vetor $v \in V$, onde $V$ é um espaço vetorial em outro espaço vetorial $W$. Para fazermos isso, precisamos realizar uma transformação em $v$, tal transformação pode ser definida por $w = pv^{t}, w \in W$.

Entendendo isso, é sabido que existe a seguinte relação:
$$W = VP$$

A pergunta então, é como encontrar R. Ora, R pode ser encontrado atravéz da descida de gradiente, técnica conhecida para minimização. Qual a função objetivo nesse caso ? 
$$Loss = ||VP - W||_{f}$$

Onde $||||_{f}$ equivale a norma de Forbenius, representada pela raiz quadrada da soma dos quadrados de todos os termos da matriz, ou $||A||_{f} = \sqrt{\sum_{i}{\sum_{j}}{a_{i,j}^{2}}}$. Agora que já sabemos a função a ser minimizada, devemos definir o gradiente da mesma. 

$$\nabla Loss = \frac{\delta Loss}{\delta P} = \frac{2 V^{t}(VP - W)}{m}$$

Agora com o algoritmo de descida de gradiente é possível encontrar P. Ainda assim, a relação $W = VP$ não é verdadeira, no caso temos a relação $W \approx VP$. O que implica que mesmo achando VP, devemos utilizar algum algoritmo que encontre a equivalência entre as palavras.

Adianto que o algoritmo usado será o KNN, e como não é nada novo pra mim, vou pular essa parte.

### Locally sensitive hashing

Esse tipo de hashing é interessante porque ao invés de agrupar os termos por modulo, ela na verdade agrupa os termos por regiões no espaço vetorial. Para definir 

$$  $$

O experimento que vou fazer aqui tem como objetivo testar esse algoritmo de tradução. Na bíblia hebraíca. kkkkk

In [216]:
import urllib.request
import urllib.parse
import re
import numpy as np

def normalize(vec):
    return vec/(np.linalg.norm(vec))
import re
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
hebrewCorpus = []
maxRange = 10
for index in range(1,maxRange):
    url = "https://biblehub.com/wlco/genesis/"+str(index)+".htm"
    response = urllib.request.urlopen(url)
    if(response.status < 300):
        contents = response.read()
        contents = contents.decode("utf-8")
        parsed_html = BeautifulSoup(contents)
        htmlTextArray = parsed_html.body.find_all('span', attrs={'class':'spcl'})
        textArray = [ele.text for ele in htmlTextArray]
        hebrewCorpus += textArray

        engCorpus = []
for index in range(1,maxRange):
    url = "https://biblehub.com/web/genesis/"+str(index)+".htm"
    response = urllib.request.urlopen(url)
    if(response.status < 300):
        contents = response.read()
        contents = contents.decode("utf-8")
        parsed_html = BeautifulSoup(contents)
        htmlTextArray = parsed_html.body.find_all('p', attrs={'class':'reg'})
        removeSpaces = [re.sub("\n|\r","",ele.text) for ele in htmlTextArray]
        textArray = []
        for text in removeSpaces:
            aux = re.sub("\d+","*",text)
            aux = aux.split("*")
            for text_ in aux:
                if text_ != '':
                    textArray += [text_]
        engCorpus += textArray
        
portugCorpus = []
for index in range(1,maxRange):
    url = "https://bibliaportugues.com/kja/genesis/"+str(index)+".htm"
    response = urllib.request.urlopen(url)
    if(response.status < 300):
        contents = response.read()
        contents = contents.decode("utf-8")
        parsed_html = BeautifulSoup(contents)
        htmlTextArray = parsed_html.body.find_all('span', attrs={'class':'maintext'})
        textArray = [ele.text for ele in htmlTextArray]
        portugCorpus += textArray

In [239]:
pontuacao = [",",".",":",";","?","!","(",")","&","“", "”","\"","'",'']
palavrasIrrelevantes = [str(line).split(" ")[0] for line in open("./palavrasIrrelevantes.txt", encoding="utf-8")]
palavrasIrrelevantesHb = [str(line) for line in open("./palavrasIrrelevantesHb.txt", encoding="utf-8")]
palavrasIrrelevantesEng = [str(line).replace("\n","") for line in open("./palavrasIrrelevantesEng.txt", encoding="utf-8")]
vocabularioPtbr = {}
vocabularioHebrew = {}
vocabularioEng = {}
maxWords = 570

def criarTokensBr(frase):
    fraseTokenizada = []
    for palavra in frase.split(" "):
        if len(palavra) == 0:
            continue
        for ponto in pontuacao:
            if ponto in palavra:
                palavra = palavra.replace(ponto,"")
        palavra = palavra.lower()
        if palavra[-1] == 's':
            palavra = palavra[:len(palavra) - 1]
        if palavra in palavrasIrrelevantes:
            continue
        fraseTokenizada.append(palavra)
    return fraseTokenizada

def criarTokensEng(frase):
    fraseTokenizada = []
    for palavra in frase.split(" "):
        if len(palavra) == 0:
            continue
        for ponto in pontuacao:
            if ponto in palavra:
                palavra = palavra.replace(ponto,"")
        palavra = palavra.lower()
        if palavra[-1] == 's':
            palavra = palavra[:len(palavra) - 1]
        if palavra in palavrasIrrelevantesEng:
            continue
        fraseTokenizada.append(palavra)
    return fraseTokenizada

def criarTokensHb(frase):
    fraseTokenizada = []
    for palavra in frase.split(" "):
        if palavra in palavrasIrrelevantesHb:
            continue
        fraseTokenizada.append(palavra)
    return fraseTokenizada

def criarVocabulario(corpo,vocabulario,criarTokens):
    index = 0
    for frase in corpo:
        fraseTokenizada = []
        fraseTokenizada = criarTokens(frase)
        for i in range((len(fraseTokenizada))):
            if fraseTokenizada[i] not in vocabulario.keys():
                vocabulario[fraseTokenizada[i]] = index
                index += 1
                if(index == maxWords):
                    return vocabulario
    return vocabulario
            
vocabularioPtbr = criarVocabulario(portugCorpus,vocabularioPtbr, criarTokensBr)
vocabularioEng = criarVocabulario(engCorpus,vocabularioEng, criarTokensEng)

In [240]:
len(vocabularioPtbr),len(vocabularioEng)

(570, 570)

In [241]:
lenVoc = len(vocabularioPtbr)
coOcBr = np.zeros((lenVoc, lenVoc))
coOcBr.shape

def knnModificado(corpus, vocab, criarTokens, matriz, k=2):
    for frase in corpus:
        tokens = criarTokens(frase)
        for i in range(len(tokens)):
            for j in range(i+1,len(tokens)):
                if(tokens[i] not in vocab.keys() or tokens[j] not in vocab.keys()):
                    continue
                if(abs(i - j) <= k):
                    matriz[vocab[tokens[i]],vocab[tokens[j]]] += 1
                    matriz[vocab[tokens[j]],vocab[tokens[i]]] += 1
                    
knnModificado(portugCorpus, vocabularioPtbr, criarTokensBr,coOcBr, k = 8)

In [242]:
coOcTarget = np.zeros((lenVoc, lenVoc))
coOcTarget.shape
knnModificado(engCorpus, vocabularioEng, criarTokensEng, coOcTarget)

In [243]:
coOcBr[0] = normalize(coOcBr[0])
for i in range(coOcBr.shape[0]):
    coOcBr[i] = normalize(coOcBr[i])
    coOcTarget[i] = normalize(coOcTarget[i])

In [244]:
W = coOcTarget
V = coOcBr 

In [245]:
def aplicarSVD(matrizCoocorrencia):
    matrizCoovariancia = (matrizCoocorrencia - np.mean(matrizCoocorrencia,axis=0))/np.std(matrizCoocorrencia,axis=0)
    print(matrizCoovariancia.shape)
    U,S,V = np.linalg.svd(matrizCoovariancia)
    return U,S,V

WU, WS, WV = aplicarSVD(W)

(570, 570)


In [246]:
VU, VS, VV = aplicarSVD(V)

(570, 570)


In [247]:
def aplicarPCA(U,S, k=100):
    return (U * S)[:,:k]
dim = 10
Wr = aplicarPCA(WU,WS, k=dim) #5034,1000
Vr = aplicarPCA(VU,VS, k=dim) #6891,1000
# W = V P ===> 5034,1000 =  5034,6891 * 6891,1000 

In [248]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

check_symmetric(V), check_symmetric(W)

(False, False)

Como ambas as matrizes são simétricas, o produto VP e PV é o mesmo. E portanto, posso realizar W = P V, resolvendo o problema de dimensionalidade pois $5034,1000 =  5034,1000 * $ é viável. Agora, devo ficar ciente que os vetores de P são expressos pela linha e não pela coluna como anotei lá em cima.

In [249]:
def decidaGradiente(W, V, lr=0.5, it= 100):
    P = np.random.normal(size=(V.shape[1], V.shape[1]))
    losses = []
    for i in range(it):
        toMin = np.dot(V,P) - W
        loss = np.linalg.norm(toMin, ord='fro')**2
        losses.append(loss)
        if(len(losses) > 2 and losses[i-1] - losses[i]<= 0.1):
            return P
        print("Iteracao",i,loss)
        grad = (np.dot(2*V.transpose(),toMin))
        grad = grad/V.shape[0]
        P = P - grad * lr        
    return P

P = decidaGradiente(W,V, it=5000,lr=.5)

Iteracao 0 329479.5058694712
Iteracao 1 311168.6387140364
Iteracao 2 295805.4931958213
Iteracao 3 282750.8944118885
Iteracao 4 271514.4508145435
Iteracao 5 261719.26685380237
Iteracao 6 253075.072476511
Iteracao 7 245357.7580374438
Iteracao 8 238393.7841970525
Iteracao 9 232048.3023390265
Iteracao 10 226216.0994639673
Iteracao 11 220814.69334368472
Iteracao 12 215779.06488469502
Iteracao 13 211057.63726905326
Iteracao 14 206609.20473593715
Iteracao 15 202400.58485291657
Iteracao 16 198404.82213803375
Iteracao 17 194599.8119925276
Iteracao 18 190967.24517791194
Iteracao 19 187491.79686993983
Iteracao 20 184160.50243345794
Iteracao 21 180962.27584633534
Iteracao 22 177887.5371922692
Iteracao 23 174927.92362857526
Iteracao 24 172076.06431498908
Iteracao 25 169325.40441873777
Iteracao 26 166670.06683638756
Iteracao 27 164104.74295803916
Iteracao 28 161624.60584502752
Iteracao 29 159225.24075112093
Iteracao 30 156902.58910551318
Iteracao 31 154652.90298208353
Iteracao 32 152472.70777077385


Iteracao 278 43123.3174085826
Iteracao 279 43028.09706810986
Iteracao 280 42933.420858163685
Iteracao 281 42839.28342202937
Iteracao 282 42745.67947474074
Iteracao 283 42652.60380189681
Iteracao 284 42560.05125850144
Iteracao 285 42468.01676782541
Iteracao 286 42376.495320290174
Iteracao 287 42285.481972373425
Iteracao 288 42194.97184553519
Iteracao 289 42104.96012516477
Iteracao 290 42015.44205954764
Iteracao 291 41926.412958851746
Iteracao 292 41837.868194133414
Iteracao 293 41749.803196361645
Iteracao 294 41662.21345546098
Iteracao 295 41575.094519372244
Iteracao 296 41488.44199313107
Iteracao 297 41402.25153796333
Iteracao 298 41316.51887039773
Iteracao 299 41231.23976139459
Iteracao 300 41146.41003549103
Iteracao 301 41062.02556996183
Iteracao 302 40978.08229399566
Iteracao 303 40894.5761878866
Iteracao 304 40811.5032822405
Iteracao 305 40728.85965719557
Iteracao 306 40646.64144165754
Iteracao 307 40564.84481254852
Iteracao 308 40483.46599406942
Iteracao 309 40402.50125697598
Iter

Iteracao 543 28198.77526946198
Iteracao 544 28163.975550815612
Iteracao 545 28129.266863294644
Iteracao 546 28094.64878450643
Iteracao 547 28060.120894945107
Iteracao 548 28025.682777965918
Iteracao 549 27991.33401975995
Iteracao 550 27957.07420932886
Iteracao 551 27922.902938460156
Iteracao 552 27888.819801702626
Iteracao 553 27854.824396342
Iteracao 554 27820.91632237685
Iteracao 555 27787.095182494944
Iteracao 556 27753.36058204954
Iteracao 557 27719.712129036212
Iteracao 558 27686.149434069797
Iteracao 559 27652.67211036159
Iteracao 560 27619.27977369673
Iteracao 561 27585.972042411937
Iteracao 562 27552.748537373463
Iteracao 563 27519.608881955184
Iteracao 564 27486.552702016994
Iteracao 565 27453.57962588338
Iteracao 566 27420.689284322358
Iteracao 567 27387.88131052436
Iteracao 568 27355.155340081677
Iteracao 569 27322.511010967817
Iteracao 570 27289.947963517207
Iteracao 571 27257.465840405195
Iteracao 572 27225.064286628083
Iteracao 573 27192.742949483458
Iteracao 574 27160.50

Iteracao 812 21183.048838876028
Iteracao 813 21163.304570231696
Iteracao 814 21143.594518111237
Iteracao 815 21123.918583807077
Iteracao 816 21104.27666903433
Iteracao 817 21084.66867592839
Iteracao 818 21065.094507042515
Iteracao 819 21045.55406534544
Iteracao 820 21026.047254219015
Iteracao 821 21006.573977455857
Iteracao 822 20987.134139257
Iteracao 823 20967.72764422956
Iteracao 824 20948.3543973845
Iteracao 825 20929.014304134238
Iteracao 826 20909.707270290477
Iteracao 827 20890.43320206186
Iteracao 828 20871.192006051817
Iteracao 829 20851.983589256284
Iteracao 830 20832.80785906149
Iteracao 831 20813.664723241807
Iteracao 832 20794.55408995754
Iteracao 833 20775.475867752775
Iteracao 834 20756.429965553227
Iteracao 835 20737.41629266411
Iteracao 836 20718.43475876802
Iteracao 837 20699.485273922855
Iteracao 838 20680.567748559664
Iteracao 839 20661.682093480635
Iteracao 840 20642.82821985703
Iteracao 841 20624.00603922709
Iteracao 842 20605.215463494063
Iteracao 843 20586.45640

Iteracao 1076 16923.7047306579
Iteracao 1077 16910.499187871195
Iteracao 1078 16897.311494227895
Iteracao 1079 16884.141611834108
Iteracao 1080 16870.989502912584
Iteracao 1081 16857.855129802363
Iteracao 1082 16844.73845495818
Iteracao 1083 16831.639440950094
Iteracao 1084 16818.558050462896
Iteracao 1085 16805.494246295737
Iteracao 1086 16792.447991361612
Iteracao 1087 16779.419248686878
Iteracao 1088 16766.40798141083
Iteracao 1089 16753.414152785146
Iteracao 1090 16740.437726173583
Iteracao 1091 16727.478665051374
Iteracao 1092 16714.53693300482
Iteracao 1093 16701.612493730852
Iteracao 1094 16688.705311036556
Iteracao 1095 16675.815348838743
Iteracao 1096 16662.94257116348
Iteracao 1097 16650.086942145685
Iteracao 1098 16637.248426028593
Iteracao 1099 16624.426987163453
Iteracao 1100 16611.62259000897
Iteracao 1101 16598.83519913091
Iteracao 1102 16586.064779201686
Iteracao 1103 16573.31129499991
Iteracao 1104 16560.574711409943
Iteracao 1105 16547.854993421523
Iteracao 1106 16535

...

Iteracao 3941 4105.809865190361
Iteracao 3942 4104.378930406937
Iteracao 3943 4102.948732747611
Iteracao 3944 4101.5192716983065
Iteracao 3945 4100.090546745399
Iteracao 3946 4098.662557375736
Iteracao 3947 4097.235303076628
Iteracao 3948 4095.8087833358377
Iteracao 3949 4094.3829976416014
Iteracao 3950 4092.95794548261
Iteracao 3951 4091.533626348015
Iteracao 3952 4090.110039727432
Iteracao 3953 4088.6871851109345
Iteracao 3954 4087.2650619890524
Iteracao 3955 4085.843669852776
Iteracao 3956 4084.4230081935543
Iteracao 3957 4083.0030765032957
Iteracao 3958 4081.5838742743617
Iteracao 3959 4080.1654009995696
Iteracao 3960 4078.7476561722005
Iteracao 3961 4077.330639285982
Iteracao 3962 4075.914349835102
Iteracao 3963 4074.4987873142
Iteracao 3964 4073.083951218374
Iteracao 3965 4071.669841043171
Iteracao 3966 4070.2564562845896
Iteracao 3967 4068.843796439087
Iteracao 3968 4067.4318610035693
Iteracao 3969 4066.020649475392
Iteracao 3970 4064.610161352367
Iteracao 3971 4063.200396132749

Iteracao 4195 3764.7238394614023
Iteracao 4196 3763.464785643206
Iteracao 4197 3762.206351983375
Iteracao 4198 3760.9485380706706
Iteracao 4199 3759.6913434942117
Iteracao 4200 3758.4347678434615
Iteracao 4201 3757.178810708241
Iteracao 4202 3755.9234716787205
Iteracao 4203 3754.668750345419
Iteracao 4204 3753.414646299209
Iteracao 4205 3752.1611591313035
Iteracao 4206 3750.908288433276
Iteracao 4207 3749.6560337970395
Iteracao 4208 3748.4043948148637
Iteracao 4209 3747.1533710793597
Iteracao 4210 3745.90296218349
Iteracao 4211 3744.653167720561
Iteracao 4212 3743.4039872842336
Iteracao 4213 3742.1554204685044
Iteracao 4214 3740.907466867726
Iteracao 4215 3739.660126076591
Iteracao 4216 3738.41339769014
Iteracao 4217 3737.167281303758
Iteracao 4218 3735.9217765131743
Iteracao 4219 3734.676882914465
Iteracao 4220 3733.4326001040454
Iteracao 4221 3732.188927678681
Iteracao 4222 3730.9458652354756
Iteracao 4223 3729.703412371877
Iteracao 4224 3728.4615686856782
Iteracao 4225 3727.22033377

Iteracao 4453 3459.3448626208847
Iteracao 4454 3458.23310809278
Iteracao 4455 3457.1218783125532
Iteracao 4456 3456.011172948603
Iteracao 4457 3454.9009916695923
Iteracao 4458 3453.7913341444596
Iteracao 4459 3452.6822000424086
Iteracao 4460 3451.573589032913
Iteracao 4461 3450.465500785714
Iteracao 4462 3449.3579349708234
Iteracao 4463 3448.2508912585185
Iteracao 4464 3447.1443693193446
Iteracao 4465 3446.0383688241177
Iteracao 4466 3444.9328894439145
Iteracao 4467 3443.827930850084
Iteracao 4468 3442.7234927142404
Iteracao 4469 3441.619574708263
Iteracao 4470 3440.5161765042976
Iteracao 4471 3439.4132977747586
Iteracao 4472 3438.310938192318
Iteracao 4473 3437.2090974299263
Iteracao 4474 3436.1077751607872
Iteracao 4475 3435.006971058372
Iteracao 4476 3433.9066847964204
Iteracao 4477 3432.8069160489313
Iteracao 4478 3431.707664490171
Iteracao 4479 3430.6089297946683
Iteracao 4480 3429.5107116372155
Iteracao 4481 3428.413009692866
Iteracao 4482 3427.315823636941
Iteracao 4483 3426.219

Iteracao 4716 3184.088527424535
Iteracao 4717 3183.10411514264
Iteracao 4718 3182.120148930924
Iteracao 4719 3181.136628520234
Iteracao 4720 3180.1535536416272
Iteracao 4721 3179.170924026367
Iteracao 4722 3178.1887394059263
Iteracao 4723 3177.2069995119846
Iteracao 4724 3176.225704076428
Iteracao 4725 3175.244852831352
Iteracao 4726 3174.2644455090567
Iteracao 4727 3173.2844818420544
Iteracao 4728 3172.304961563055
Iteracao 4729 3171.325884404982
Iteracao 4730 3170.347250100963
Iteracao 4731 3169.3690583843345
Iteracao 4732 3168.3913089886337
Iteracao 4733 3167.4140016476067
Iteracao 4734 3166.437136095205
Iteracao 4735 3165.4607120655824
Iteracao 4736 3164.484729293105
Iteracao 4737 3163.5091875123344
Iteracao 4738 3162.5340864580467
Iteracao 4739 3161.55942586521
Iteracao 4740 3160.585205469011
Iteracao 4741 3159.6114250048317
Iteracao 4742 3158.638084208259
Iteracao 4743 3157.6651828150852
Iteracao 4744 3156.692720561306
Iteracao 4745 3155.7206971831192
Iteracao 4746 3154.749112416

Iteracao 4978 2940.6646177627695
Iteracao 4979 2939.7884543403466
Iteracao 4980 2938.912673041715
Iteracao 4981 2938.037273646101
Iteracao 4982 2937.1622559328944
Iteracao 4983 2936.2876196816383
Iteracao 4984 2935.41336467205
Iteracao 4985 2934.5394906840083
Iteracao 4986 2933.6659974975446
Iteracao 4987 2932.792884892862
Iteracao 4988 2931.920152650322
Iteracao 4989 2931.0478005504497
Iteracao 4990 2930.175828373932
Iteracao 4991 2929.304235901615
Iteracao 4992 2928.433022914511
Iteracao 4993 2927.5621891937867
Iteracao 4994 2926.6917345207767
Iteracao 4995 2925.821658676974
Iteracao 4996 2924.9519614440337
Iteracao 4997 2924.082642603771
Iteracao 4998 2923.21370193816
Iteracao 4999 2922.3451392293387


In [250]:
palavras = np.dot(V,P)
palavras.shape

(570, 570)

In [251]:
reverseVocabularioEng = dict([(value,key) for key,value in vocabularioEng.items()])
def traduzir(palavra,P,Vr,Wr):
    tokens = criarTokensBr(palavra)
    if(len(tokens) == 0):
        return ""
    palavra = tokens[0]
    if palavra not in vocabularioPtbr.keys():
        return "Indefinido"
    projecao = np.dot(Vr[vocabularioPtbr[palavra]],P)
    dist = 0
    index = 0
    for key, value in vocabularioEng.items():
        if key not in vocabularioEng.keys():
            return "Indefinido"
        newdist = np.dot(normalize(projecao),normalize(Wr[vocabularioEng[key],:]))
        if(dist < newdist):
            dist = newdist
            index = key 
    return index

traduzir("deus",P,V,W)
#np.dot(normalize(palavras[3]),normalize(Wr[vocabularioPtbr["senhor"],:]))

'god'

In [ ]:
def traduzirFrase(frase):
    result = ""
    print("\n" + frase)
    for palavra in frase.split(" "):
        val = traduzir(palavra,P,V,W)
        if len(val) != 0:
            result += val + " "    
    print(result)
    
for verso in portugCorpus:
    traduzirFrase(verso)



No princípio, Deus criou os céus e a terra.
beginning god created corrupt earth 

A terra, entretanto, era sem forma e vazia. A escuridão cobria o mar que envolvia toda a terra, e o Espírito de Deus se movia sobre a face das águas.
earth wa formles empty darknes surface deep spirit man earth water god said let light saw 

Disse Deus: “Haja luz!”, e houve luz.
good god divided called day called 

Viu Deus que a luz era boa; e separou a luz das trevas.
night god called fifth morning called one 

Chamou Deus à luz “Dia”, e às trevas chamou “Noite”. Houve então, a tarde e a manhã: o primeiro dia.
expanse god called middle one expanse divide day made sky second gathered middle 

Depois disse Deus: “Haja entre as águas um limite para separá-las em duas partes!”
together good god divided saw place dry land appear 

Fez, portanto, Deus o firmamento e separou as águas estabelecidas abaixo desse limite, das que ficaram por cima. E assim aconteceu.
gathering sea god put morning saw forth gras he

walking serpent god cool hid presence among afraid told eaten done array four deceived 

O SENHOR Deus fez nascer do solo toda espécie de árvores agradáveis aos olhos e boas para alimento. E no meio desse jardim estavam a árvore da vida e a árvore do conhecimento do bem e do mal.
eye god gathering cursed desired man multiply move belly go fifth shall enmity herb hid thing move alone move offspring bruise heel 

Na região do Éden nascia um rio que irrigava todo o jardim e depois se dividia em quatro.
presence among greatly pain childbirth creature hid together children desire 

O nome do primeiro é Pisom. Ele percorre todo o território de Havilá, onde existe ouro.
bearing gathered adam listened creature sake toil thorn thistle sweat 

O ouro daquela terra é de pureza excelente; terra na qual se encontra o bdélio, raro perfume, e a valiosa pedra ônix.
sweat face earth bread return earth eve coat skin ashamed become hand take 

O segundo, que percorre toda a terra de Cuxe, é chamado Giom.

room finish cain upward side desired lower earth level even know fig fig earth flood 

Assim, Adão deu à sua mulher o nome de Eva, porquanto ela seria mãe de toda a humanidade
fruit killed god happen bearing establish father fallen built man naked 

Fez Yahweh Deus túnicas de pele e com elas vestiu Adão e Eva, sua mulher.
gathering serpent god covenant sort alive killed establish happen 

Então declarou Yahweh Deus: “Eis que agora o ser humano tornou-se como um de nós, conhecendo o bem e o mal. Não devemos permitir que ele também estenda a sua mão e tome do fruto da árvore da vida e comendo-o possa viver para sempre!”
made u serpent god keep gather four thu handle according bruise heel household seen righteousnes pair clean winged move alone cause six hundredth month 

Por isso o SENHOR expulsou o ser humano do jardim do Éden e fez que ele lavrasse a terra da qual havia sido formado.
eye seventeenth four hid among gathering fountain earth burst even 

Deus baniu Adão e Eva e no lado le

man Indefinido alone Indefinido Indefinido Indefinido hi together Indefinido 

Quando Cainã completou setenta anos, gerou Maalaleel.
Indefinido Indefinido Indefinido hi Indefinido Indefinido 

Depois do nascimento de Maalaleel, Cainã viveu oitocentos e quarenta anos, e gerou filhos e filhas.
together Indefinido Indefinido Indefinido Indefinido Indefinido Indefinido hi Indefinido end Indefinido 

Toda a duração da vida de Cainã foi de novecentos e dez anos; depois morreu.
man Indefinido alone Indefinido Indefinido Indefinido hi together Indefinido 

Quando Maalaleel completou sessenta e cinco anos, gerou Jarede.
Indefinido Indefinido Indefinido Indefinido hi Indefinido Indefinido 

Depois do nascimento de Jarede, Maalaleel viveu oitocentos e trinta anos, e gerou filhos e filhas.
together Indefinido Indefinido Indefinido Indefinido Indefinido Indefinido hi Indefinido end Indefinido 

Toda a duração da vida de Maalaleel foi de oitocentos e noventa e cinco anos; depois morreu.
man Indefini

knowledge multiply yet knowledge multiply mother knowledge multiply creature river desired Indefinido Indefinido Indefinido Indefinido alone 

Quanto a ti, reúne todo tipo de alimento e armazena-o; isso servirá de alimento para ti e para eles.
Indefinido heart Indefinido creature bird shall Indefinido Indefinido shall heart 

Noé assim fez; tudo em conformidade com o que Deus lhe ordenara, ele concluiu a obra.
Indefinido fruit gathering gave whole god Indefinido Indefinido closed 

Então o SENHOR ordenou a Noé: “Entra na arca, tu e toda a tua família, porque és o único justo que observo diante de mim no meio desta geração.
made eye gihon Indefinido Indefinido Indefinido man blood Indefinido know Indefinido Indefinido Indefinido Indefinido rainbow enmity pas Indefinido 

De todos os animais puros, tomarás sete pares, o macho e sua fêmea; dos animais que não são puros, tomarás apenas um casal, o macho e sua fêmea;
creature mother Indefinido Indefinido Indefinido Indefinido onyx stone mot

gathered middle gathered Indefinido hi Indefinido alone Indefinido Indefinido saw Indefinido earth Indefinido made Indefinido Indefinido Indefinido night knew earth enosh great 

No vigésimo sétimo dia do segundo mês, a terra já estava bem seca.
Indefinido took middle third Indefinido earth enosh bruise great 

Então ordenou Deus a Noé:
made gihon god Indefinido 

“Sai da tua arca, tu e tua esposa, teus filhos e as esposas de teus filhos contigo.
Indefinido blood Indefinido blood Indefinido end Indefinido end Indefinido 

Todos os animais que estão contigo; todas as criaturas, as aves, os grandes animais e os pequenos, todos que se movem rente ao solo. Faze-os deixar a arca contigo: que pululem sobre a face da terra, sejam fecundos e multipliquem-se sobre toda a terra!”
creature mother Indefinido man back yet given mother havilah creature gold aromatic desired Indefinido Indefinido Indefinido Indefinido Indefinido let light earth male breath Indefinido let man earth 

Assim, Noé e sua 

In [79]:
def ladoDoPlano(P,v):
    prod = np.dot(P,v.transpose())
    sinais = np.sign(prod)
    return sinais

def hashSensivel(P,v):
    valorHash = 0
    sinais = ladoDoPlano(P,v)
    sinais = (sinais >= 0) * 1
    index = np.sum(2**np.array(range(P.shape[0])) * sinais)
    return index

def criarHashing(P,V,vocabulario):
    reverseVocabulario = dict([(value,key) for key, value in vocabulario.items()])
    hashing = {}
    wordHashing = {}
    for i in range(V.shape[0]):
        vec = V[i]
        index = hashSensivel(P,vec)
        if index in hashing:
            hashing[index].append(vec)
            wordHashing[index].append(reverseVocabulario[i])
        else:
            hashing[index] = [vec]
            wordHashing[index] = [reverseVocabulario[i]]
    return hashing, wordHashing


In [187]:
engCorpus

['In the beginning God created the heavens and the earth. ',
 "Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the waters.",
 'God said, "Let there be light," and there was light. ',
 'God saw the light, and saw that it was good. God divided the light from the darkness. ',
 'God called the light "day," and the darkness he called "night." There was evening and there was morning, one day.',
 'God said, "Let there be an expanse in the middle of the waters, and let it divide the waters from the waters." ',
 'God made the expanse, and divided the waters which were under the expanse from the waters which were above the expanse; and it was so. ',
 'God called the expanse "sky." There was evening and there was morning, a second day.',
 'God said, "Let the waters under the sky be gathered together to one place, and let the dry land appear;" and it was so. ',
 'God called the dry land "earth," and the gathering together